In [ ]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py transformers torch einops atlassian-python-api

In [1]:
import os 
from dotenv import load_dotenv
from getpass import getpass

load_dotenv()

LANGCHAIN_TRACING_V2 = os.environ['LANGCHAIN_TRACING_V2'] = 'true'
LANGCHAIN_ENDPOINT = os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
FIRECRAWL_API_KEY = os.environ['FIRECRAWL_API_KEY']
NOTION_API_KEY = os.environ['NOTION_API_KEY']
NOTION_DB_ID = os.environ['NOTION_DB_ID']
JIRA_API_KEY = os.environ['JIRA_API_KEY']
ATLASSIAN_USERNAME = os.environ["JIRA_USERNAME"]
ATLASSIAN_URL = os.environ["JIRA_INSTANCE_URL"]
JIRA_CLOUD_ID = os.environ["JIRA_CLOUD_ID"]

In [2]:
from langchain_community.chat_models import ChatOllama
from langchain.agents import AgentType, initialize_agent
from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit
from langchain_community.utilities.jira import JiraAPIWrapper

llm = ChatOllama(model="llama3.1:latest", format="json", temperature=0)

In [3]:
# Initialize the Jira API wrapper
jira = JiraAPIWrapper(
    jira_api_token=JIRA_API_KEY,
    jira_instance_url=ATLASSIAN_URL,
    jira_cloud=JIRA_CLOUD_ID
)

# Create the toolkit from the Jira API wrapper
toolkit = JiraToolkit.from_jira_api_wrapper(jira)

# Initialize the agent with the toolkit and LLM, enabling parsing error handling
agent = initialize_agent(
    toolkit.get_tools(),
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True  # Enable error handling to retry on parsing errors
)

/Users/josuesanchez/Personal Project/chatbot/myenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
agent.invoke("make a new issue in project PW to remind me to make more fried rice")



> Entering new AgentExecutor chain...
{
  "action": "Create Issue",
  "action_input": {
    "summary": "Make More Fried Rice",
    "description": "Reminder to make more delicious fried rice for dinner",
    "issuetype": {"name": "Task"},
    "priority": {"name": "Low"},
    "project": {"key": "PW"}
  }
}

> Finished chain.


{'input': 'make a new issue in project PW to remind me to make more fried rice',
 'output': '{\n  "action": "Create Issue",\n  "action_input": {\n    "summary": "Make More Fried Rice",\n    "description": "Reminder to make more delicious fried rice for dinner",\n    "issuetype": {"name": "Task"},\n    "priority": {"name": "Low"},\n    "project": {"key": "PW"}\n  }\n}'}

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import FireCrawlLoader
from langchain.docstore.document import Document
from langchain_community.document_loaders import NotionDBLoader

# Load documents with firecrawl a web scraper tool
urls = [
    "https://www.tokyotechies.com",
    "https://www.tokyotechies.com/about-us",
    "https://www.tokyotechies.com/solutions/kotae"
]
docs = [FireCrawlLoader(api_key=FIRECRAWL_API_KEY, url=url, mode="scrape").load() for url in urls]


# Flatten the list of documents
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Filter out complex metadata
filtered_docs = []
for doc in doc_splits:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# Save the filtered_docs to a file or cache for later use
import pickle

with open('filtered_docs.pkl', 'wb') as f:
    pickle.dump(filtered_docs, f)

In [ ]:
import pickle 

with open('filtered_docs.pkl', 'rb') as f:
    filtered_docs = pickle.load(f)

print(filtered_docs)

In [5]:
from transformers import AutoModel, AutoTokenizer
import torch
from langchain_community.vectorstores import Chroma

# Load the filtered_docs from the saved file
import pickle

with open('filtered_docs.pkl', 'rb') as f:
    filtered_docs = pickle.load(f)

# Load the embedding model and tokenizer
model_name = "nomic-ai/nomic-embed-text-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to generate embeddings using the loaded model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].cpu().numpy().tolist()  # Convert ndarray to list

# Wrapper class to use with Chroma
class CustomEmbedding:
    def embed_documents(self, texts):
        return [embed_text(text) for text in texts]

    def embed_query(self, text):
        return embed_text(text)

# Instantiate the embedding class
custom_embedding = CustomEmbedding()

# Add documents with embeddings to the vectorDB using the embedding class
vectorstore = Chroma.from_documents(
    documents=filtered_docs,
    collection_name="rag-chroma",
    embedding=custom_embedding,  # Use the embedding class instance
)

retriever = vectorstore.as_retriever()



/Users/josuesanchez/Personal Project/chatbot/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/Users/josuesanchez/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/e55a7d4324f65581af5f483e830b80f34680e8ff/modeling

In [6]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Define the prompt template
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing 
    relevance of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n 
    Give a binary score 'yes' or 'no' score to indicate whether the docuemnt is relevant to the question. \n 
    Providde the binary score as a JSON with a single key 'score' and no premable or explaination.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document* \n\n {document} \n\n
    Here is the user question* {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
    input_variables=["question", "document"]
)

# Chain the prompt, LLM, and output parser together
retrieval_grader = prompt | llm | JsonOutputParser()

# Define the user question
question = "What is Tokyo Techies?"

# Retrieve documents related to the question
docs = retriever.invoke(question)

# Get the content of the second retrieved document
doc_txt = docs[1].page_content  # Use page_content instead of page_context

# Grade the relevance of the document
result = retrieval_grader.invoke({"question": question, "document": doc_txt})

# Print the result
print(result)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'score': 'yes'}


In [18]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assitant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
input_variables=["question", "document"]
)

#Post processing
def format_doc(docs):
    return"\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

#Run

question = "What do you know about kotae?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

{ "Kotae is a chatbot platform that allows small businesses to automate conversations and delight customers. It can be trained using a company's knowledge base, website scrapes, training files, and FAQs." 

  





  





  





  





  





  





  





  





  





  








In [19]:
### Hallucination Grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing whether
    an answer is grounded in / supported by a set of facts. Give binary scores 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preambel or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n -------- \n
    {documents}
    \n -------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
input_variables=["generation", "document"]
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [20]:
### Answer grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing whether
    the answer is useful in resolve a question. Give binary scores 'yes' or 'no' score to indicate 
    whether the answer is use to resolve a question. Provide the binary score as a JSON with a 
    single key 'score' and no preambel or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the answer:
    \n -------- \n
    {generation}
    \n -------- \n
    Here is the question: {question}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
input_variables=["generation", "question"]
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}